## Library

In [6]:
import os
import json
import time
import requests
import pandas as pd
import numpy as np
import re
import unicodedata
from dotenv import load_dotenv
from datetime import datetime, timezone


## CALL API 

In [7]:
load_dotenv()
NOTION_KEY = os.getenv("Notion_Key")
DATABASE_ID = os.getenv("DATABASE_ID")

headers = {
    "Authorization": f"Bearer {NOTION_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

# (optionnel) vérifier l'accès à la DB
meta = requests.get(f"https://api.notion.com/v1/databases/{DATABASE_ID}", headers=headers)
print("meta:", meta.status_code, meta.text[:300])

# requête
resp = requests.post(f"https://api.notion.com/v1/databases/{DATABASE_ID}/query",
                     headers=headers, json={})
print("query:", resp.status_code, resp.text[:500])


meta: 200 {"object":"database","id":"17e1db20-4c05-819a-b255-fed182b63251","cover":null,"icon":null,"created_time":"2025-01-17T16:29:00.000Z","created_by":{"object":"user","id":"17ed872b-594c-81f2-b2df-0002e3a50029"},"last_edited_by":{"object":"user","id":"17ed872b-594c-81f2-b2df-0002e3a50029"},"last_edited_t
query: 200 {"object":"list","results":[{"object":"page","id":"2411db20-4c05-809b-8915-cf8d57a6ca9c","created_time":"2025-07-31T20:19:00.000Z","last_edited_time":"2025-07-31T20:19:00.000Z","created_by":{"object":"user","id":"17ed872b-594c-81f2-b2df-0002e3a50029"},"last_edited_by":{"object":"user","id":"17ed872b-594c-81f2-b2df-0002e3a50029"},"cover":null,"icon":{"type":"emoji","emoji":"🤖"},"parent":{"type":"database_id","database_id":"17e1db20-4c05-819a-b255-fed182b63251"},"archived":false,"in_trash":false,"


In [8]:

load_dotenv()
NOTION_KEY = os.getenv("Notion_Key")
DATABASE_ID = os.getenv("DATABASE_ID")

HEADERS = {
    "Authorization": f"Bearer {NOTION_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

def rich_text_to_str(rts):
    return "".join((r or {}).get("plain_text","") for r in (rts or []))

def prop_to_value(prop):
    t = prop.get("type")
    v = prop.get(t)
    if t == "title":         return rich_text_to_str(v)
    if t == "rich_text":     return rich_text_to_str(v)
    if t == "number":        return v
    if t == "select":        return v["name"] if v else None
    if t == "multi_select":  return ", ".join(x["name"] for x in (v or []))
    if t == "status":        return v["name"] if v else None
    if t == "date":          return (v or {}).get("start")
    if t == "checkbox":      return bool(v)
    if t == "people":        return ", ".join((p.get("name") or p.get("id")) for p in (v or []))
    if t == "email":         return v
    if t == "phone_number":  return v
    if t == "url":           return v
    if t == "files":         return ", ".join(f.get("name") for f in (v or []))
    if t == "relation":      return ", ".join(item.get("id") for item in (v or []))
    if t == "rollup":
        # simplification: on renvoie du JSON lisible
        return json.dumps(v, ensure_ascii=False)
    return json.dumps(v, ensure_ascii=False)

def fetch_all(database_id, page_size=100):
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    body = {"page_size": page_size}
    has_more, start_cursor = True, None
    results = []
    while has_more:
        if start_cursor:
            body["start_cursor"] = start_cursor
        r = requests.post(url, headers=HEADERS, json=body)
        r.raise_for_status()
        data = r.json()
        results.extend(data.get("results", []))
        has_more = data.get("has_more", False)
        start_cursor = data.get("next_cursor")
        time.sleep(0.2)  # léger throttle
    return results

def pages_to_rows_props_only(pages):
    rows = []
    for pg in pages:
        props = pg.get("properties", {})
        row = {}
        for name, prop in props.items():
            row[name] = prop_to_value(prop)
        rows.append(row)
    return rows

# --- EXTRACT ---
pages = fetch_all(DATABASE_ID)
print(f"Pages récupérées: {len(pages)}")

# --- TRANSFORM ---
rows = pages_to_rows_props_only(pages)
df = pd.DataFrame(rows)

# --- LOAD (CSV uniquement) ---
os.makedirs("data", exist_ok=True)
csv_path = "data/suivi_candidatures.csv"
df.to_csv(csv_path, index=False)
print(f"Écrit: {csv_path}")


Pages récupérées: 281
Écrit: data/suivi_candidatures.csv


## Load Data + Cleaning - V1 to V2

In [15]:
df=pd.read_csv('/Users/fredericmendessemedo/Desktop/Data - Next Step/PP1_Recherche_emploi/data/suivi_candidatures.csv')
df.head(10)

,Poste,Date de Soumission,État,Lien,Date de réponse,CV utilisé,Contact Email,Entreprise
0,Data Analyst Environnement,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,NaN,NaN,Inex Circular
1,Data Engineer Junior,2025-07-31,Candidature envoyée,https://apply.workable.com/singulier/j/C258FD8...,NaN,NaN,NaN,Singulier
2,Data Analyst,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,NaN,NaN,ARCEP
3,Data scientist,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,NaN,NaN,Viginum
4,Data Scientist - Artifical Intelligence Projec...,2025-07-31,Refus,https://ekez.fa.em2.oraclecloud.com/hcmUI/Cand...,2025-08-01,NaN,NaN,Natixis
5,"Data Quality Specialist, AI Tutor - Paris (Fix...",2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,NaN,NaN,Mistral AI
6,CDI - Data Analyst - Petit Bateau !,2025-07-31,Candidature envoyée,https://www.linkedin.com/jobs/view/4274098673/...,NaN,NaN,NaN,Petit Bateau
7,Assistant Gestion Référentiels et Outils H/F,2025-07-31,Candidature envoyée,https://jobs.totalenergies.com/en_US/careers/A...,NaN,NaN,NaN,Total Energies
8,\nGestionnaire de données GED,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,CV3,NaN,RESAH
9,CDD - Analyste Surveillance des Engagements F/H,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,CV3,NaN,La Banque Postale


In [16]:
df['Date de Soumission']=pd.to_datetime(df['Date de Soumission'],dayfirst=False)
df['Mois'] = df['Date de Soumission'].dt.month

In [18]:
#Nettoyer les sauts de ligne dans Poste / Entreprise
for col in ["Poste", "Entreprise"]:
    if col in df.columns:
        df[col] = (
            df[col].astype(str)
                   .str.replace(r"\\n|/n|\r|\n", " ", regex=True)  # gère "\n" ET "/n"
                   .str.replace(r"\s+", " ", regex=True)
                   .str.strip()
        )

# Date de réponse = Soumission + 30 jours si Etat == "Sans réponse"
df["Date de Soumission"] = pd.to_datetime(df["Date de Soumission"], errors="coerce", dayfirst=False)
# Si la colonne existe, normaliser, sinon la créer vide
if "Date de réponse" in df.columns:
    df["Date de réponse"] = pd.to_datetime(df["Date de réponse"], errors="coerce")
else:
    df["Date de réponse"] = pd.NaT

etat_sr = (
    df["État"]
      .astype(str)
      .str.normalize("NFKC")
      .str.strip()
      .str.lower()
      .eq("sans réponse")
)
df.loc[etat_sr, "Date de réponse"] = df.loc[etat_sr, "Date de Soumission"] + pd.Timedelta(days=30)

# "CV utilisé" : partir de CV1 puis propager CV2, CV3… chronologiquement ---

if "CV utilisé" in df.columns:
    # Trier par date pour respecter la chronologie (garde un index pour re-trier si besoin)
    df["_idx"] = np.arange(len(df))
    df = df.sort_values("Date de Soumission", kind="stable")

    # Extraire un éventuel numéro de CV (CV2, CV 3, cv4...)
    cv_num = (
        df["CV utilisé"].astype(str)
                        .str.extract(r"[Cc][Vv]\s*([0-9]+)", expand=False)
    )
    explicit_cv = "CV" + cv_num
    explicit_cv = explicit_cv.where(cv_num.notna(), np.nan)

    # Forward-fill à partir des mentions explicites, baseline = CV1
    df["CV utilisé"] = explicit_cv.ffill().fillna("CV1")

    # Revenir à l’ordre initial si tu veux
    df = df.sort_values("_idx").drop(columns="_idx")
else:
    # Si la colonne n'existe pas, on en crée une à CV1 partout
    df["CV utilisé"] = "CV1"

In [20]:
def strip_accents(s: str) -> str:
    """Supprime les accents et met en minuscule."""
    if not isinstance(s, str):
        return ""
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn").lower()

def categorize_poste(poste: str) -> str:
    """
    Retourne la catégorie du poste à partir du texte (FR + EN).
    Catégories: Data Analyst, Data Scientist, Data Engineer, Analytics Engineer, Consultant Data, Autre
    """
    text = strip_accents(poste)

    # --- Analytics Engineer ---
    if re.search(r"\banalytics engineer\b", text) or ("analytics" in text and "engineer" in text):
        return "Analytics Engineer"
    if "ingenieur analytics" in text:
        return "Analytics Engineer"

    # --- Data Engineer ---
    if "data engineer" in text or "ingenieur donnees" in text or "ingenieur data" in text:
        return "Data Engineer"
    if any(word in text for word in ["etl", "elt", "pipeline", "big data", "spark", "hadoop", "airflow", "databricks", "warehouse", "snowflake", "redshift", "bi engineer", "mlops"]):
        return "Data Engineer"

    # --- Data Scientist ---
    if "data scientist" in text or "scientifique des donnees" in text or "science des donnees" in text:
        return "Data Scientist"
    if any(word in text for word in ["machine learning", "deep learning", "intelligence artificielle", "ia ", "ml engineer", "nlp", "computer vision", "statisticien", "research scientist"]):
        return "Data Scientist"

    # --- Data Analyst ---
    if "data analyst" in text or "analyste" in text:
        return "Data Analyst"
    if any(word in text for word in ["charge d etudes", "reporting", "power bi", "tableau", "looker", "metabase", "business analyst", "bi analyst", "marketing analyst"]):
        return "Data Analyst"

    # --- Consultant Data ---
    if "consultant data" in text or "consultant en donnees" in text or "consultant ia" in text:
        return "Consultant Data"
    if any(word in text for word in ["consultant bi", "consultant analytics", "consultant big data", "consultant digital", "consultant decisionnel"]):
        return "Consultant Data"

    return "Autre"

# Application au DataFrame
df["Catégorie Poste"] = df["Poste"].apply(categorize_poste)


In [23]:
df["Catégorie Poste"].value_counts()

Catégorie Poste
Autre                 81
Data Analyst          75
Data Scientist        60
Data Engineer         42
Analytics Engineer    17
Consultant Data        6
Name: count, dtype: int64

In [24]:
csv_path = "data/suivi_candidatures_V2.csv"
df.to_csv(csv_path, index=False)
print(f"Écrit: {csv_path}")

Écrit: data/suivi_candidatures_V2.csv
